In [66]:
from database_handling import DBHandling

In [67]:
# Utilisation de gradio pour l'app avec SQLite + Docker ? https://www.youtube.com/watch?v=-GgG1Wb2F64&list=PLi0b6yHwHZcGaIy4ZGkYLZqssvzwqkBia&index=19&ab_channel=MASTER2SISEDATASCIENCE
# https://www.sqlite.org/docs.html
# Avantage -> Juste un pacakge, pas besoin d'installer un serveur, + facile pour déploiement docker
# Acces avec des fichiers et des tables à l'intérieurs (pysqlite doit être installé avec C++, CMake etc -> Trop compliqué avec docker, on va utiliser pysqlite3 wheels -> version déjà compilée)
# Compatbile avec streamlit mais démo avec gradio
# Design pattern https://www.youtube.com/watch?v=BJatgOiiht4&ab_channel=ForrestKnight
## TODO faire le sqlite + ajouter des champs dans le scrapping(clef étrangère)
"""
- Reste Insertion Restaurants + Avis
- Delete Restaurants + Avis
- Update Restaurant + Avis
- Requêtes particulières ( restaurants avec plus de 4 étoilse etc...)

"""
# Changer la BDD pour chaque arro,dissement depuiss le champ adress ### OK
# Créer une table arrondissement 
# https://www.insee.fr/fr/statistiques/1405599?geo=COM-69123+COM-69383+COM-69384+COM-69385+COM-69386+COM-69387+COM-69388+COM-69389+COM-69382+COM-69381

'\n- Reste Insertion Restaurants + Avis\n- Delete Restaurants + Avis\n- Update Restaurant + Avis\n- Requêtes particulières ( restaurants avec plus de 4 étoilse etc...)\n\n'

In [68]:
db = DBHandling("../../data/database.db")
# connection et création des tables
db.connect()
db.create_tables()
# help(db)


Connexion à la base de données réussie : ../../data/database.db
Tables créées avec succès.


In [69]:
# test selection
rows = db.fetch_all("SELECT * FROM restaurants")
rows

[]

In [70]:
import os
import pandas as pd

###################################### RUN ONLY ONCE ######################################

data_directory = "../../data"
description_directory = "../../data/descriptions"
all_restaurants = []
all_descriptions = []

# Read restaurant info CSV files
for filename in os.listdir(data_directory):
    filepath = os.path.join(data_directory, filename)

    if filename.endswith("_infos.csv"):
        try:
            df = pd.read_csv(filepath, encoding='utf-8', usecols=["Nom", "Type_Cuisine", "Fourchette_prix", "Adresse", "Note_moyenne"], index_col=False)
            all_restaurants.append(df)
        except pd.errors.EmptyDataError:
            print(f"Le fichier {filename} est vide")
        except Exception as e:
            print(f"Erreur lors du traitement de {filename}: {e}")

# Read description CSV files
for filename in os.listdir(description_directory):
    filepath = os.path.join(description_directory, filename)

    if filename.endswith("_description.csv"):
        try:
            df = pd.read_csv(filepath, encoding='utf-8', usecols=["Nom", "Description"], index_col=False)
            all_descriptions.append(df)
        except pd.errors.EmptyDataError:
            print(f"Le fichier {filename} est vide")
        except Exception as e:
            print(f"Erreur lors du traitement de {filename}: {e}")

# Concatenate all restaurant info DataFrames
if all_restaurants:
    all_restaurants_df = pd.concat(all_restaurants, ignore_index=True)

# Concatenate all description DataFrames
if all_descriptions:
    all_descriptions_df = pd.concat(all_descriptions, ignore_index=True)


In [71]:
all_restaurants_df.head(1)

,Nom,Type_Cuisine,Fourchette_prix,Adresse,Note_moyenne
0,Crêperie Amélie,"Réservations, Places assises, Chaises hautes d...",NaN,"112 Cours Charlemagne, 69002 Lyon France","2,5"


In [72]:
all_descriptions_df.head(1)

,Nom,Description
0,Crêperie Amélie,NaN


In [73]:
if not all_restaurants_df.empty and not all_descriptions_df.empty:
    merged_df = pd.merge(all_restaurants_df, all_descriptions_df, on="Nom", how="left")

    # Insert each row of the merged DataFrame into the database
    for _, row in merged_df.iterrows():
        try:
            db.insert_restaurant(
                nom=row["Nom"],
                type_cuisine=row["Type_Cuisine"],
                fourchette_prix=row["Fourchette_prix"],
                adresse=row["Adresse"],
                note_moyenne=row["Note_moyenne"],
                description=row["Description"]
            )
            print(f"Restaurant ajouté : {row['Nom']}")
        except KeyError as e:
            print(f"Erreur dans le fichier {filename}: colonne manquante {e}")
       

112, Cours Charlemagne, Confluence, Lyon 2e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69002, France
Insertion réussie.
Restaurant ajouté : Crêperie Amélie
23, Rue de Condé, Ainay, Lyon 2e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69002, France
Insertion réussie.
Restaurant ajouté : Frazarin Bistrot Franco Italien
106, Rue du Président Édouard Herriot, Bellecour, Lyon 2e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69002, France
Insertion réussie.
Restaurant ajouté : Pizza Pino Lyon
43, Quai Rambaud, Confluence, Lyon 2e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69002, France
Insertion réussie.
Restaurant ajouté : Selcius
Rue Antoine Delandine, Sainte-Blandine, Lyon 2e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69002, France
Insertion réussi

In [74]:
# # Parcourir les fichiers CSV dans le répertoire
# import os
# import csv

# ###################################### RUN ONLY ONCE ######################################

# data_directory = "../../data"
# for filename in os.listdir(data_directory):
#     filepath = os.path.join(data_directory, filename)

#     if filename.endswith("_infos.csv"):
#         # Traitement des fichiers contenant les informations des restaurants
#         with open(filepath, newline='', encoding='utf-8') as csvfile:
#             reader = csv.DictReader(csvfile)
#             for row in reader:
#                 try:
#                     db.insert_restaurant(
#                         nom=row["Nom"],
#                         type_cuisine=row["Type_Cuisine"],
#                         fourchette_prix=row["Fourchette_prix"],
#                         adresse=row["Adresse"],
#                         note_moyenne=row["Note_moyenne"],
#                         description=
#                     )
#                     print(f"Restaurant ajouté : {row['Nom']}")
#                 except KeyError as e:
#                     print(f"Erreur dans le fichier {filename}: champ manquant {e}")


In [75]:
rows_restau = db.fetch_all("SELECT * FROM restaurants")
rows_restau

[(1,
  'Crêperie Amélie',
  "Réservations, Places assises, Chaises hautes disponibles, Accessible en fauteuil roulant, Sert de l'alcool, Service de table",
  None,
  '112 Cours Charlemagne 69002 Lyon France',
  '2,5',
  None,
  69002,
  45.7406889,
  4.818801),
 (2,
  'Frazarin Bistrot Franco Italien',
  'Italienne, Française, Saine',
  '16,00\xa0€-38,00\xa0€',
  '23 Rue De Condé 69002 Lyon France',
  '5,0',
  "FRAZARIN est un Bistrot Franco - Italien qui vous amène à découvrir l'histoire imaginaire de FRAZARIN, jeune Italien que nous suivons dans son voyage initiatique à travers la France, curieux de découvrir les secrets et techniques des maîtres Auguste Escoffier, Paul Bocuse ou Henri Babinski. La cuisine du bistrot est à l'image de ce jeune homme : fier de ses racines italiennes et désireux de découvrir toute la profondeur et la complexité de la cuisine française, riche de ses terroirs. Dans le but final d'une belle rencontre entre le verre et l'assiette, chez FRAZARIN, de l'entrée

In [76]:
# Insertion des avis
# test avec affreux_jojo_avis.csv, ID restaurant 1
import csv
def insert_avis(chemin:str, clef:int):
    with open(chemin, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            try:
                db.insert_avis(
                    id_restaurant=clef,
                    date = row["Date"],
                    note=row["Notes"],
                    commentaire=row["Commentaires"]
                )
                # print(f"Avis ajouté : {row['Commentaires']}")
            except KeyError as e:
                print(f"Erreur dans le fichier {filename}: champ manquant {e}")

# insert_avis("../../data/affreux_jojo_avis.csv", 1)
# insert_avis("../../data/chez_michel_avis.csv", 2)
# insert_avis("../../data/chez_michel_avis.csv", 3)
# insert_avis("../../data/chez_michel_avis.csv", 4)
# insert_avis("../../data/chez_michel_avis.csv", 5)
# insert_avis("../../data/chez_michel_avis.csv", 6)
# insert_avis("../../data/chez_michel_avis.csv", 7)
# insert_avis("../../data/chez_michel_avis.csv", 8)
# insert_avis("../../data/chez_michel_avis.csv", 9)
# insert_avis("../../data/chez_michel_avis.csv", 10)

data_directory = "../../data"
cpt = 1
for filename in os.listdir(data_directory):
    filepath = os.path.join(data_directory, filename)

    if filename.endswith("_avis.csv"):
        # print(filename)
        insert_avis(filepath, cpt)
        cpt += 1

Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion ré

In [77]:
rows_avis = db.fetch_all("SELECT * FROM avis")
rows_avis

[(1,
  1,
  'déc. 2024',
  '2,0',
  "quantité: garniture e ma galette minable: 4 mini morceaux de jambon, 5 rondelles de champignons, 1 jaune d'oeuf quasi cru, pas de blanc d'oeuf\nqualité: aucun goût\nprix: cher pour quasiment rien à manger (14,90€)\n\nsalle pas chauffée, bon accueil"),
 (2,
  1,
  'déc. 2024',
  '5,0',
  'Juste après un cinéma, déjeuner en famille avec deux enfants pour le menu enfant. Note contenue pour un repas très agréable et un service très rapide. Une chouette adresse !'),
 (3,
  1,
  'nov. 2024',
  '2,0',
  'Restaurant sans intérêt, la crêpe salée est très médiocre trop cuite et pas très garnie mais la crêpe sucrée est encore pire pas assez cuite trop épaisse. Passez votre chemin surtout si vous aimez les crêpes.'),
 (4,
  1,
  'oct. 2024',
  '2,0',
  'On a demandé la crêpe du jour...ah bah pas pendant les vacances !! \nComme cela quand on vient en famille c\'est plus cher...\nEt quand je demande pourquoi, la réponse de la serveuse ,: "c\'est comme ça".\nSi je

In [78]:
# Select les avis correspondant au restaurant 10
rows_avis_10 = db.fetch_all("SELECT * FROM avis WHERE id_restaurant = 10")
rows_avis_10

[(4326,
  10,
  'déc. 2024',
  '5,0',
  'Pizza excellente, bonne ambiance, très bon service, au top. On a demandé 2 jojo et 1 pizza et tout a été parfait.\r\n\r\nMerci bcpp!'),
 (4327,
  10,
  'déc. 2024',
  '5,0',
  "C'est l'endroit parfait à Lyon pour manger une pizza napolitaine tout à fait délicieuse!\r\nAmbiance et personnel très sympa!"),
 (4328,
  10,
  'déc. 2024',
  '5,0',
  'Nous avons dégustés d’excellentes pizze.\r\nUne pizza diavola juste piquante comme il faut et une pizza au jambon truffé, à la crème de truffe parsemé de buratta.\r\nUn repas carrément divin !!!'),
 (4329,
  10,
  'nov. 2024',
  '4,0',
  'Tres bel endroit en famille on mange très bien..je conseille...ce restaurant tres belle équipe.et une bonne ambiance'),
 (4330,
  10,
  'nov. 2024',
  '5,0',
  'Très bonne pizza bien gourmande et pâtes du jour délicieuse.\r\nDéco et cadre agréable \r\nAccueil très souriant.\r\nJe recommande sans hésiter.'),
 (4331,
  10,
  'nov. 2024',
  '5,0',
  "Une pizza ou un panozzo

In [79]:
restau_69008 = db.fetch_all("SELECT * FROM restaurants WHERE arrondissement = 69008")
restau_69002 = db.fetch_all("SELECT * FROM restaurants WHERE arrondissement = 69002")

print(len(restau_69008))
print(len(restau_69002))

5
6


In [80]:
db.close()

In [81]:
# Test Regex

import re

def extract_postal_code(address: str) -> int:
    """
    Extrait le code postal d'une adresse.

    Args:
        address (str): L'adresse complète.

    Returns:
        int: Le code postal extrait ou None si non trouvé.
    """
    # Regex pattern to match French postal codes (5 digits)
    pattern = r'\b\d{5}\b'
    match = re.search(pattern, address)
    if match:
        return int(match.group(0))
    return None

# Example usage
address = "1 rue de Paris, 75001 Paris"
postal_code = extract_postal_code(address)
print(f"Code postal extrait: {postal_code}")

Code postal extrait: 75001


### Open Data :

In [ ]:
# PIMP21	Part des mÃ©nages fiscaux imposÃ©s en 2021	Part des mÃ©nages fiscaux imposÃ©s en 2021			CHAR	4
# MED21	MÃ©diane du niveau vie en 2021	MÃ©diane du niveau de vie en 2021			CHAR	5
# TP6021	Taux de pauvretÃ© en 2021	Taux de pauvretÃ© en 2021			CHAR	4


In [88]:
col_names = ["CODGEO", "PIMP21", "MED21", "TP6021"]
df_insee = pd.read_csv("../../data/DataInsee/base_cc_comparateur.csv", sep=";", encoding='utf-8', usecols=col_names)
df_insee.head(10)

,CODGEO,PIMP21,MED21,TP6021
0,69381,60,25980,15
1,69382,66,28920,12
2,69383,65,26920,14
3,69384,66,28240,11
4,69385,63,26130,13
5,69386,71,33050,9
6,69387,60,24410,16
7,69388,52,21100,23
8,69389,54,21650,22
